In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json
import gc

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

import cv2

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
BASE_PATH = '/kaggle/input/cassava-leaf-disease-classification/'
TRAIN_IMAGE_PATH = BASE_PATH + 'train_images'
TEST_IMAGE_PATH = BASE_PATH + 'test_images'

SEED_VAL = 42
IMG_SHAPE = 512
BATCH_SZ = 16

In [ ]:
#set seed value for reproducibility 
tf.random.set_seed(SEED_VAL)
np.random.seed(SEED_VAL)

### Step 1 => Data Reading and Preparation

In [ ]:
train_df = pd.read_csv(BASE_PATH + 'train.csv')
print(train_df.shape)
print(train_df.head())

In [ ]:
#check the distribution of images for each label
sns.countplot(x='label', data=train_df)
plt.show()

In [ ]:
#read the label file to know the categories
labels_dict = dict()

with open(BASE_PATH + 'label_num_to_disease_map.json') as file_handle:
    labels_dict = json.load(file_handle)

print(labels_dict)

In [ ]:
#display a couple of images from dataset
fig,ax = plt.subplots(1,5,sharey=True, figsize=(20,6))
for i in range(5):
    image = train_images[i,:,:,:]
    label = train_labels[i]
    ax[i].imshow(image)
    ax[i].set_title(labels_dict[str(label)])

plt.show()

In [ ]:
train_df['label'] = train_df['label'].astype('str')

train_generator = ImageDataGenerator(rescale=1./255, 
                                     validation_split=0.2,
                                    zoom_range = 0.2,
                                    horizontal_flip = True,
                                     vertical_flip = True,
                                    height_shift_range = 0.2,
                                     width_shift_range = 0.2).flow_from_dataframe(train_df, 
                                    directory=TRAIN_IMAGE_PATH, subset='training', x_col='image_id',
                                    y_col='label', target_size=(IMG_SHAPE, IMG_SHAPE), batch_size = BATCH_SZ,
                                    class_mode='sparse')

val_generator = ImageDataGenerator(rescale=1./255, validation_split=0.2).flow_from_dataframe(train_df, 
                                    directory=TRAIN_IMAGE_PATH, subset='validation', x_col='image_id',
                                    y_col='label', target_size=(IMG_SHAPE, IMG_SHAPE), batch_size = BATCH_SZ,
                                    class_mode='sparse')

### Step 2 - Model Building

In [ ]:
#Define a CNN

#Define CNN
model = tf.keras.models.Sequential([
          tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_SHAPE,IMG_SHAPE,3)),
          tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
          
          tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
          tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
          
          tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
          tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
          
          tf.keras.layers.Dropout(0.2),
          
          tf.keras.layers.Flatten(),    

          tf.keras.layers.Dense(512, activation='relu'),
          tf.keras.layers.Dropout(0.2),
          tf.keras.layers.Dense(5, activation='softmax')
        ])

model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.001, mode='min', patience=10,
                              restore_best_weights=True)

model_checkpoint = ModelCheckpoint('./best_model.h5', save_best_only=True, save_weights_only=True, 
                                  verbose=1, monitor='val_loss', mode='min')

In [ ]:
history = model.fit_generator(train_generator,
          steps_per_epoch=(len(train_df)*0.8)/BATCH_SZ,
          epochs=20,
          validation_data=val_generator,
          validation_steps = (len(train_df)*0.2)/BATCH_SZ,
        callbacks=[early_stopping, model_checkpoint])                    

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(10,8))

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.subplot(1,2,1)
ax[0].plot(range(1,6), acc, label='training accuracy')
ax[0].plot(range(1,6), val_acc, label='validation accuracy')

plt.subplot(1,2,2)
ax[1].plot(range(1,6), loss, label='training loss')
ax[1].plot(range(1,6), val_loss, label='validation loss')

plt.show()